![banner](https://bccfilkom.net/static/assets/images/BCC-Logo.svg)

# Day 2
### BCC 2023

> Evan \
> Biyu

## Review Day 1

## Intro Machine Learning

### Apa itu ML?

### Glossarium

### Tipe Machine Learning

Berdasarkan Cara Belajar

1. Supervised
2. Unsupervised
3. dll..

Berdasarkan task-nya
1. Regresi
2. Klasifikasi

## Data Preprocessing

### Scaling

### Feature Transformation

### Split

## Regresi

### Animasi

### Training

### Evaluasi

## Klasifikasi

### Animasi

### Training

### Evaluasi

## Interpret Model

### SHAP

### Apakah menjawab masalah?

## Tugas 😨